In [ ]:
import railrl.misc

In [ ]:
%matplotlib inline

In [2]:
4+4

8

# Create a basic RNN

In [1]:
import tensorflow as tf
import numpy as np

In [9]:
sess = tf.InteractiveSession()

In [7]:
num_steps = 3
x_dim = 2
y_dim = 2
state_size = 5
learning_rate = 1e-3

In [8]:
x = tf.placeholder(
    tf.int32,
    [None, num_steps, num_classes],
    name='input_placeholder',
)
y = tf.placeholder(
    tf.int32,
    [None, num_steps, num_classes],
    name='labels_placeholder',
)

rnn_inputs = tf.unpack(tf.cast(x, tf.float32), axis=1)
labels = tf.unpack(tf.cast(y, tf.float32), axis=1)

cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
rnn_outputs, final_state = tf.nn.rnn(
    cell,
    rnn_inputs,
    dtype=tf.float32,
)

with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [state_size, num_classes])
    b = tf.get_variable('b', [num_classes],
                        initializer=tf.constant_initializer(0.0))
logits = [tf.matmul(rnn_output, W) + b for rnn_output in rnn_outputs]
predictions = [tf.nn.softmax(logit) for logit in logits]

total_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits, labels)
train_step = tf.train.AdamOptimizer(
    learning_rate).minimize(
    total_loss)

In [10]:
print(cell.output_size)
print(cell.state_size)

5
LSTMStateTuple(c=5, h=5)


In [ ]:
cell_input = tf.placeholder(
    tf.int32,
    [None, ]
)
cell_input = tf.placeholder(
    tf.int32,
    [None]
)
state = np.array([0, 0, 0, 0, 0]), np.array([0, 0, 0, 0, 0])
cell_output = cell(cell_input, cell_state)
sess.run(
    cell
)